## 벡터 연산과 Numpy로 그림 그리기

이번 프로젝트에서는 Numpy를 이용한 벡터 연산으로 그림을 그려보겠습니다.

#### 캔버스

그림을 그리기 위해서는 그림을 그릴 공간이 필요합니다. 이 프로젝트에서는 이 공간을 `xrange`, `yrange` 라는 변수로 지정하겠습니다. 만약,

```
xrange = [1, 3]
yrange = [2, 4]
```

라면, 그림을 그릴 캔버스는 $(1, 2)$, $(3, 4)$ 로 지정된 공간을 사용하게 됩니다.

#### 그림 그리기

그림을 그리기 위해서, 다음 방식을 사용하겠습니다. 어떤 함수 `f`와 매우 작은 숫자 `threshold`에 대해, 

 * 캔버스 내에 점 `P = (x, y)`을 임의로 생성한다.
 * `f(P) < threshold`  라면 점을 찍는다. 만약 그렇지 않다면, 점을 찍지 않는다.
 * 이것을 100,000 회 반복한다.
 
왜 `f(P) == 0` 일때 점을 찍지 않고, 아주 작은 값 `threshold` 보다 작을 때 점을 찍는지, 한번 생각해 보세요!

## 예제: 원 그리기

이 예제는 `circle` 함수에 정의되어 있습니다.

$(0, 0)$ 이 중심이고 반지름 1인 원을 그리는 방정식은 다음과 같습니다.

$$x^2 + y^2 = 1$$

위의 그림을 그리는 방식을 생각하면, 정확히 원 위에 있는 점들에 대해서 `circle(P)` 은 0을 가져야 합니다. 그러므로, `circle(P)` 는 다음과 같이 정의할 수 있습니다.

```python
x = P[0]
y = P[1]
return sqrt(x ** 2 + y ** 2) - 1
```

이것은 수업시간에 배운 것과 같이 다음과 같습니다.

```python
return sqrt(np.sum(P * P)) - 1
```

## 프로젝트

예제로 주어진 원 `circle` 과 다이아몬드 `diamond` 함수를 이용해 원과 다이아몬드 그림을 그려봅니다. 그리고, 이것들을 조합해 새로운 그림을 만듭니다. 조합하는 방법은 `smile` 함수를 참고합니다.

In [ ]:
import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pyplot as plt
import elice_utils
import numpy as np
elice = elice_utils.EliceUtils()

def circle(P):
    return np.linalg.norm(P) - 1 # 밑의 코드와 동일하게 동작합니다.
    # return np.sqrt(np.sum(P * P)) - 1
    
def diamond(P):
    return np.abs(P[0]) + np.abs(P[1]) - 1
    
def smile(P):
    def left_eye(P):
        eye_pos = P - np.array([-0.5, 0.5])
        return np.sqrt(np.sum(eye_pos * eye_pos)) - 0.1
    
    def right_eye(P):
        eye_pos = P - np.array([0.5, 0.5])
        return np.sqrt(np.sum(eye_pos * eye_pos)) - 0.1
    
    def mouth(P):
        if P[1] < 0:
            return np.sqrt(np.sum(P * P)) - 0.7
        else:
            return 1
    
    return circle(P) * left_eye(P) * right_eye(P) * mouth(P)

def checker(P, shape, tolerance):
    return abs(shape(P)) < tolerance

def sample(num_points, xrange, yrange, shape, tolerance):
    accepted_points = []
    rejected_points = []
    
    for i in range(num_points):
        x = np.random.random() * (xrange[1] - xrange[0]) + xrange[0]
        y = np.random.random() * (yrange[1] - yrange[0]) + yrange[0]
        P = np.array([x, y])
        
        if (checker(P, shape, tolerance)):
            accepted_points.append(P)
        else:
            rejected_points.append(P)
    
    return np.array(accepted_points), np.array(rejected_points)

xrange = [-1.5, 1.5] # X축 범위입니다.
yrange = [-1.5, 1.5] # Y축 범위입니다.
accepted_points, rejected_points = sample(
    100000, #  점의 개수를 줄이거나 늘려서 실행해 보세요. 너무 많이 늘리면 시간이 오래 걸리는 것에 주의합니다.
    xrange, 
    yrange, 
    smile, # smile을 circle 이나 diamond 로 바꿔서 실행해 보세요.
    0.005) # Threshold를 0.01이나 0.0001 같은 다른 값으로 변경해 보세요.

plt.figure(figsize=(xrange[1] - xrange[0], yrange[1] - yrange[0]), 
           dpi=150) # 그림이 제대로 로드되지 않는다면 DPI를 줄여보세요.
           
plt.scatter(rejected_points[:, 0], rejected_points[:, 1], c='lightgray', s=0.1)
plt.scatter(accepted_points[:, 0], accepted_points[:, 1], c='black', s=1)

plt.savefig("graph.png")
elice.send_image("graph.png")